<a href="https://colab.research.google.com/github/tmohammad78/deep-learning-projects/blob/feature%2Fvariational-autoencoder-mnist/variational-autoencoder/mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf

In [7]:
def encoder_layers(inputs,latent_dim):
  x = tf.keras.layers.Conv2D(filters=32,padding="same",name="encoder_conv1" , activation="relu" , kernel_size=3 , strides=2)(inputs)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(filters=64,padding="same",name="encoder_conv2" , activation="relu" , kernel_size=3 , strides=2)(x)
  batch_2 = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Flatten(name="encoder_flatten")(batch_2)
  x = tf.keras.layers.Dense(20,activation="relu",name="encode_dense")(x)
  x = tf.keras.layers.BatchNormalization()(x)

  mu = tf.keras.layers.Dense(latent_dim,name="latent_mu")(x)
  sigma = tf.keras.layers.Dense(latent_dim,name="latent_sigma")(x)
  return mu , sigma , batch_2.shape

In [8]:
class Sampling(tf.keras.layers.Layer):
  def call(self,inputs):
    mu , sigma = inputs
    batch = tf.shape(mu)[0]
    dim = tf.shape(mu)[1]
    eplison = tf.keras.backend.random_normal(shape=(batch,dim))
    return mu + tf.exp(0.5 * sigma) * spsilon
    

In [9]:
def encoder_model(LATENT_DIM,input_shape):
  inputs = tf.keras.layers.Input(input=input_shape)
  mu , sigma , conv_shape = encoder_layers(inputs , latent_dim=LATENT_DIM)
  z = Sampling()((mu,sigma))
  model = tf.keras.Model(inputs,outputs=[mu,sigma,z])
  return model,conv_shape

In [10]:
def decoder_layers(inputs,conv_shape):
  units = conv_shape[1] * conv_shape[2] * conv_shape[3]
  x = tf.keras.layer.Dense(units,activation="relu",name="decoder_dens1")(inputs)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Reshape((conv_shape[1],conv_shape[2],conv_shape[3]),name="decoder_shape")(x)
  x = tf.keras.layers.Conv2DTranspose(filters=64,kernel_size=3,strides=2,padding="same",activation="relu" , name="decoder_conv2d_2")(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding="same",activation="relu" , name="decoder_conv2d3")(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(filters=1,kernel_size=3,strides=1,padding="same",activation="sigmoid" , name="decoder_final")(x)
  return x

In [11]:
def decoder_model(latent_dim,conv_shape):
   inputs = tf.keras.layers.Input(shape=(latent_dim,))
   outputs = decoder_layers(inputs,conv_shape)
   model = tf.keras.Model(inputs,outputs)
   return model

In [12]:
def k1_reconstrcution_loss(input,output,mu , sigma):
  kl_loss = 1 + sigma - tf.square(mu) - tf.math.exp(sigma)
  return tf.reduce_mean(kl_loss) * -0.5


In [14]:
def vae_model(encoder, decoder , input_shape):
  inputs = tf.keras.layers.Inputs(shape=(input_shape))
  mu = encoder(inputs)[0]
  sigma = encoder(inputs)[1]
  z = encoder(inputs)[2]
  reconstructed = decoder(z)
  model = tf.keras.Model(inputs=inputs,outputs = reconstructed)
  loss = k1_reconstrcution_loss(inputs,z , mu , sigma)
  model.add_loss(loss)
  return model

